## Importación y carga de dataloaders

In [1]:
import pickle
import glob
import torch
from torch.utils.data import DataLoader, Dataset
from eeg_fConn import connectivity as con
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

class ConnectivityDataset(Dataset):
    def __init__(self, original_dataloader, f_min, f_max, fs, sensors):
        self.original_dataloader = original_dataloader
        self.f_min = f_min
        self.f_max = f_max
        self.fs = fs
        self.sensors = sensors
        self.pli = [] 
        self.plv = []  
        self.ccf = []  
        self.coh = []  
        self.labels = []  
        self.ages = []

        for batch in original_dataloader:
            signal = batch['signal'].squeeze().cpu()
            label = batch['class_label'].squeeze(dim=0).cpu()
            age = batch['age'].squeeze(dim=0).cpu()
            
            filtered_data = con.filteration(data=signal, f_min=f_min, f_max=f_max, fs=fs)
            Mi, _ = con.pli_connectivity(sensors, data=filtered_data)
            self.pli.append(Mi)
            Mv, _ = con.plv_connectivity(sensors, data=filtered_data)
            self.plv.append(Mv)
            Mf, _ = con.ccf_connectivity(sensors, data=filtered_data)
            self.ccf.append(Mf)
            Mh, _ = con.coh_connectivity(sensors, data=signal, f_min=f_min, f_max=f_max, fs=fs)
            
            self.coh.append(Mh)
            self.labels.append(label)
            self.ages.append(age)

    def __len__(self):
        return len(self.original_dataloader)

    def __getitem__(self, index):
        pli_matrix = self.pli[index]
        plv_matrix = self.plv[index]
        ccf_matrix = self.ccf[index]
        coh_matrix = self.coh[index]
        label = self.labels[index]
        age = self.ages[index]

        pli_matrix = torch.tensor(pli_matrix)
        plv_matrix = torch.tensor(plv_matrix)
        ccf_matrix = torch.tensor(ccf_matrix)
        coh_matrix = torch.tensor(coh_matrix)
        label = torch.tensor(label)
        age = torch.tensor(age)

        return {'pli': pli_matrix, 'plv': plv_matrix, 'ccf': ccf_matrix, 'coh': coh_matrix, 'label': label, 'age': age}

# Dataloader de entrenamiento
with open('../dataloaders/con_dataloader_train.pkl', 'rb') as file:
    dataloader = pickle.load(file)
    
# Dataloader de validación    
with open('../dataloaders/con_dataloader_val.pkl', 'rb') as file:
    dataloader_val = pickle.load(file)


cuda:0


# Construcción modelo LSTM 

Se construyó un modelo tipo LSTM para probar su eficacia en este tipo de datos. Se implementaron también distintos bloques para el aumento de la profundidad del modelo.

In [4]:
class Bottleneck(nn.Module):
    def __init__(self, input_size, bottleneck_size):
        super(Bottleneck, self).__init__()
        self.fc1 = nn.Linear(input_size, bottleneck_size)
        self.activation1 = nn.ReLU()
        self.fc2 = nn.Linear(bottleneck_size, bottleneck_size)
        self.activation2 = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation1(x)
        x = self.fc2(x)
        x = self.activation2(x)
        return x


class StackedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, layer_size, bottleneck_size, output_size, bidirectional=True, dropout=0.5):
        super(StackedLSTM, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.layer_size = layer_size
        self.bottleneck_size = bottleneck_size
        self.output_size = output_size
        self.bidirectional = bidirectional
        self.dropout = dropout
        
        self.bottleneck = Bottleneck(input_size, bottleneck_size)

        self.lstms = nn.ModuleList()
        for i in range(layer_size):
            if i == 0:
                input_dim = bottleneck_size
            else:
                input_dim = hidden_size * 2 if bidirectional else hidden_size
            lstm = nn.LSTM(input_dim, hidden_size, 1, batch_first=True, bidirectional=bidirectional)
            self.lstms.append(lstm)
            self.lstms.append(nn.Dropout(dropout))

        if bidirectional:
            self.fc = nn.Linear(hidden_size * 2, output_size)
        else:
            self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, label_ts):
        batch_size = label_ts.size(0)

        label_ts = label_ts.view(batch_size, -1, self.input_size)

        label_ts = self.bottleneck(label_ts)

        hidden_states = []
        cell_states = []
        for i in range(self.layer_size):
            if self.bidirectional:
                hidden_state = torch.zeros(2, batch_size, self.hidden_size).to(label_ts.device)
                cell_state = torch.zeros(2, batch_size, self.hidden_size).to(label_ts.device)
            else:
                hidden_state = torch.zeros(1, batch_size, self.hidden_size).to(label_ts.device)
                cell_state = torch.zeros(1, batch_size, self.hidden_size).to(label_ts.device)
            hidden_states.append(hidden_state)
            cell_states.append(cell_state)

        output = label_ts
        for i in range(self.layer_size):
            lstm = self.lstms[i * 2] 
            dropout = self.lstms[i * 2 + 1]
            output, (hidden_state, cell_state) = lstm(output, (hidden_states[i], cell_states[i]))
            output = dropout(output)
            hidden_states[i] = hidden_state

        output = output[:, -1, :]
        output = self.fc(output)

        return output


#=========================================INSTANCIA DEL MODELO========================================

input_size = 19 
hidden_size = 128
output_size = 3
bottleneck_size = 64
layer_size = 2
dropout = 0.5

model = StackedLSTM(input_size, hidden_size, layer_size, bottleneck_size, output_size, dropout=dropout)
model.to(device)

criterion = nn.CrossEntropyLoss()

learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model)

StackedLSTM(
  (bottleneck): Bottleneck(
    (fc1): Linear(in_features=19, out_features=64, bias=True)
    (activation1): ReLU()
    (fc2): Linear(in_features=64, out_features=64, bias=True)
    (activation2): ReLU()
  )
  (lstms): ModuleList(
    (0): LSTM(64, 128, batch_first=True, bidirectional=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): LSTM(256, 128, batch_first=True, bidirectional=True)
    (3): Dropout(p=0.5, inplace=False)
  )
  (fc): Linear(in_features=256, out_features=3, bias=True)
)


## Entrenamiento y evaluación del modelo

In [5]:
from tqdm import tqdm


train_accu = []
train_losses = []
eval_losses = []
eval_accu = []

def train(num_epochs):
    print('\nEpoch: %d' % num_epochs)
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for data in tqdm(dataloader):
        reduced_matrix = data['coh'].unsqueeze(0).float().to(device)
        class_label = data['label'].squeeze().unsqueeze(0).to(torch.long).to(device)
        
        age = data['age']
        
        optimizer.zero_grad()
        outputs = model(reduced_matrix)
        loss = criterion(outputs, class_label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = outputs.max(1)
        total += class_label.size(0)
        correct += predicted.eq(class_label).sum().item()

    train_loss = running_loss / len(dataloader.dataset)
    accu = 100. * correct / total

    train_accu.append(accu)
    train_losses.append(train_loss)
    print('Train Loss: %.3f | Accuracy: %.3f' % (train_loss, accu))

def test(epoch):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data in tqdm(dataloader_val):
            reduced_matrix = data['coh'].unsqueeze(0).float().to(device)
            class_label = data['label'].squeeze().unsqueeze(0).to(torch.long).to(device)

            outputs = model(reduced_matrix)

            loss = criterion(outputs, class_label)
            running_loss += loss.item()

            _, predicted = outputs.max(1)
            total += class_label.size(0)
            correct += predicted.eq(class_label).sum().item()

    test_loss = running_loss / len(dataloader_val.dataset)
    accu = 100. * correct / total

    eval_losses.append(test_loss)
    eval_accu.append(accu)

    print('Test Loss: %.3f | Accuracy: %.3f' % (test_loss, accu)) 

epochs = 30
for epoch in range(1, epochs + 1): 
    train(epoch)
    test(epoch)

# Imprimir resultados finales
print('Train Losses:', train_losses)
print('Train Accuracy:', train_accu)
print('Test Losses:', eval_losses)
print('Test Accuracy:', eval_accu)



Epoch: 1


  0%|                                                                                          | 0/950 [00:00<?, ?it/s]C:\Users\hernandez\AppData\Local\Temp\ipykernel_4912\114427283.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label)
C:\Users\hernandez\AppData\Local\Temp\ipykernel_4912\114427283.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  age = torch.tensor(age)
100%|███████████████████████████████████████████████████████████████████████████████| 950/950 [00:05<00:00, 164.45it/s]


Train Loss: 1.097 | Accuracy: 36.316


100%|███████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 481.96it/s]


Test Loss: 1.086 | Accuracy: 38.655

Epoch: 2


...

Test Loss: 1.055 | Accuracy: 42.857

Epoch: 30


100%|███████████████████████████████████████████████████████████████████████████████| 950/950 [00:06<00:00, 156.44it/s]


Train Loss: 0.979 | Accuracy: 49.579


100%|███████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 424.59it/s]

Test Loss: 1.082 | Accuracy: 47.059
Train Losses: [1.0969942392800984, 1.0939241913745277, 1.094021969719937, 1.090655310153961, 1.081487930134723, 1.071115386140974, 1.069223765558318, 1.0518032852128931, 1.0581265476130342, 1.0505637136023296, 1.0430259500522363, 1.0430500937586553, 1.0361857782696422, 1.0294822721457795, 1.026060837103348, 1.0274496212444808, 1.0273587284472427, 1.0259541302684105, 1.019152924524326, 1.013458189697642, 1.0078485709075864, 1.0080080372487243, 1.0115093919576, 0.9967451778485586, 0.9947454738165987, 0.9868023733637835, 0.9972910904158887, 0.9913525615750175, 0.9883160705237012, 0.9785525072699315]
Train Accuracy: [36.31578947368421, 35.36842105263158, 37.36842105263158, 38.63157894736842, 38.63157894736842, 39.68421052631579, 41.68421052631579, 41.89473684210526, 40.21052631578947, 42.8421052631579, 45.578947368421055, 45.578947368421055, 45.68421052631579, 48.421052631578945, 47.05263157894737, 47.578947368421055, 45.78947368421053, 47.47368421052631

In [7]:
torch.save(model.state_dict(), 'LSTM.pth')